In [28]:
import json
import os
import sys
import uuid
from dotenv import load_dotenv
from azure.core.exceptions import AzureError
from azure.cosmos import CosmosClient, PartitionKey, exceptions

In [29]:
env_path = ".env"
load_dotenv(env_path)

True

In [30]:
ENDPOINT = os.environ["COSMOS_ENDPOINT"]
KEY = os.environ["COSMOS_KEY"]

client = CosmosClient(url=ENDPOINT, credential=KEY)

In [31]:
DATABASE = "transcriptions-db"
CONTAINER_TRANSCRIPTION = 'transcriptions-container'
CONTAINER_METADADOS = 'metadata-container'
CONTAINER_LOGS = 'logs-container'

In [32]:
database = client.get_database_client(DATABASE)
container_transcription = database.get_container_client(CONTAINER_TRANSCRIPTION)

In [33]:
# Função para carregar dados de um arquivo JSON
def carregar_dados_json(caminho_arquivo):
    try:
        with open(caminho_arquivo, 'r', encoding='utf-8') as file:
            return json.load(file)
    except FileNotFoundError:
        print(f"Arquivo {caminho_arquivo} não encontrado.")
    except json.JSONDecodeError as e:
        print(f"Erro ao decodificar o JSON: {e}")
    return None

In [34]:
# Função para inserir transcrições no contêiner
def inserir_transcricoes(container, data):
    try:
        for item in data['transcription']:
            transcricao_item = {
                'audio_path': data['audio_path'],  # chave de partição
                'order': item['order'],
                'start': item['start'],
                'end': item['end'],
                'transcription': item['transcription']
            }
            container.create_item(body=transcricao_item)
        print("Transcrições inseridas com sucesso!")
    except exceptions.CosmosHttpResponseError as e:
        print(f"Erro ao inserir transcrições: {e}")

# Função para inserir métricas no contêiner
def inserir_metricas(container, data):
    try:
        metrics_item = {
            'audio_path': data['audio_path'],  # chave de partição
            'total_words': data['metrics']['total_words'],
            'words_per_minute': data['metrics']['words_per_minute'],
            'top_10_words': data['metrics']['top_10_words'],
            'magic_word_percentages': data['metrics']['magic_word_percentages']
        }
        container.create_item(body=metrics_item)
        print("Métricas inseridas com sucesso!")
    except exceptions.CosmosHttpResponseError as e:
        print(f"Erro ao inserir métricas: {e}")

In [35]:
# Criar os contêineres se não existirem
transcriptions_container = database.create_container_if_not_exists(
    id="TranscriptionsContainer",
    partition_key=PartitionKey(path=r"C:\Users\sasde\azure-connection-cosmosdb\json_files 1"),
    offer_throughput=400
)

metrics_container = database.create_container_if_not_exists(
    id="MetricsContainer",
    partition_key=PartitionKey(path=r"C:\Users\sasde\azure-connection-cosmosdb\json_files 1"),
    offer_throughput=400
)

In [36]:
# Carregar dados do arquivo JSON
caminho_arquivo_json = r"C:\\Users\\sasde\\azure-connection-cosmosdb\\json_files" 
data = carregar_dados_json(caminho_arquivo_json)

if data:
    # Inserir dados nos contêineres
    inserir_transcricoes(transcriptions_container, data)
    inserir_metricas(metrics_container, data)
else:
    print("Não foi possível carregar os dados do arquivo JSON.")

PermissionError: [Errno 13] Permission denied: 'C:\\\\Users\\\\sasde\\\\azure-connection-cosmosdb\\\\json_files 1'